### Домашнее задание №2 к курсу "Рекомендательные системы"

In [ ]:
!pip install implicit==0.4.4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving retail_train.csv to retail_train (1).csv


# Новый раздел

In [ ]:
import io

In [ ]:
data =pd.read_csv(io.BytesIO(uploaded['retail_train.csv']))
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [ ]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [ ]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    
        
    items = np.array(items_weights['item_id'])   # вектор items_id
    P = np.array(items_weights['weight']) #вектор весов
    recs = np.random.choice(items, size=n, replace=False, p=P)     
    
    return recs.tolist()

In [ ]:
%%time
result = pd.DataFrame()
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']

CPU times: user 174 ms, sys: 17.8 ms, total: 192 ms
Wall time: 205 ms


In [ ]:
result

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412..."
...,...,...
2037,2496,[6534178]
2038,2497,"[1016709, 9835695, 1132298, 16809501, 845294, ..."
2039,2498,"[15716530, 834484, 901776, 914190, 958382, 972..."
2040,2499,"[867188, 877580, 902396, 914190, 951590, 95813..."


In [ ]:
sales_value_sum = popular['sales_value'].sum()
popular['weight'] = popular['sales_value']/sales_value_sum
popular

,item_id,sales_value,weight,weight_log
0,25671,20.94,2.969296e-06,3.041661
1,26081,0.99,1.403822e-07,-0.010050
2,26093,1.59,2.254623e-07,0.463734
3,26190,1.54,2.183723e-07,0.431782
4,26355,1.98,2.807644e-07,0.683097
...,...,...,...,...
86859,17330511,9.98,1.415166e-06,2.300583
86861,17382205,7.99,1.132983e-06,2.078191
86862,17383227,4.49,6.366828e-07,1.501853
86863,17827644,2.50,3.545005e-07,0.916291


In [ ]:
popular = popular[popular['weight'] != 0]
popular

,item_id,sales_value,weight,weight_log
0,25671,20.94,2.969296e-06,3.041661
1,26081,0.99,1.403822e-07,-0.010050
2,26093,1.59,2.254623e-07,0.463734
3,26190,1.54,2.183723e-07,0.431782
4,26355,1.98,2.807644e-07,0.683097
...,...,...,...,...
86859,17330511,9.98,1.415166e-06,2.300583
86861,17382205,7.99,1.132983e-06,2.078191
86862,17383227,4.49,6.366828e-07,1.501853
86863,17827644,2.50,3.545005e-07,0.916291


In [ ]:
%%time
result['random_recommendation_wight'] = result['user_id'].apply(lambda x: weighted_random_recommendation(popular, n=5))


CPU times: user 4.08 s, sys: 126 ms, total: 4.21 s
Wall time: 5.15 s


In [ ]:
result.head(2)

,user_id,actual,random_recommendation_wight
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[995965, 1134688, 1115098, 1124029, 848071]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1062002, 845208, 908318, 906844, 6534178]"


## Задание 2. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая случаный на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [ ]:
#Попробуйте улучшить бейзлайны, считая случаный на топ-5000 товаров
popular5000 = popular.sort_values('sales_value', ascending=False)[:5000]

In [ ]:
sales_value_sum = popular5000['sales_value'].sum()
popular5000['weight'] = popular5000['sales_value']/sales_value_sum

In [ ]:
result['random_recommendation_5000'] = result['user_id'].apply(lambda x: weighted_random_recommendation(popular5000, n=5))

In [ ]:
result.head(5)

,user_id,actual,random_recommendation_wight,random_recommendation_5000
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[995965, 1134688, 1115098, 1124029, 848071]","[856252, 1022003, 1070803, 6534178, 1068708]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1062002, 845208, 908318, 906844, 6534178]","[951590, 6533765, 918368, 1056492, 845307]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1139901, 6534178, 1036249, 12731419, 16809497]","[901067, 1011300, 7168058, 1014732, 1025611]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[911622, 952163, 917033, 992986, 949835]","[9365106, 6534178, 951164, 13381928, 6533765]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[988202, 1056509, 982009, 1135917, 872342]","[5569327, 6534178, 6034964, 1011312, 863505]"


----------------------

In [ ]:
#Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей  K .
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

popularity.head()

,item_id,n_sold
0,25671,6
1,26081,1
2,26093,1
3,26190,1
4,26355,2


In [ ]:
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [ ]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix)

In [ ]:
# создаем словари мапинга между id бизнеса к строчному id матрицы

userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [ ]:
#fit
%%time
K=5
model = ItemItemRecommender(K=K, num_threads=4) # K - кол-во билжайших соседей


model.fit(sparse_user_item.T.tocsr(),  # На вход item-user matrix
          show_progress=True)


recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)

  0%|          | 0/86865 [00:00<?, ?it/s]

CPU times: user 24.1 s, sys: 210 ms, total: 24.4 s
Wall time: 17.2 s


In [ ]:
recs

[(34707, 75614.0),
 (23590, 60381.0),
 (36454, 28271.0),
 (39619, 21721.0),
 (25064, 18600.0)]

In [ ]:
%%time

result['itemitem5'] = result['user_id'].apply(lambda user_id: [id_to_itemid[rec[0]]  
                                                              for rec in model.recommend(userid=userid_to_id[user_id], 
                                                                user_items=sparse_user_item,   # на вход user-item matrix
                                                                N=5, 
                                                                filter_already_liked_items=False, 
                                                                filter_items=None, 
                                                                recalculate_user=True)
                                                                             ])

CPU times: user 133 ms, sys: 1.93 ms, total: 135 ms
Wall time: 135 ms


In [ ]:
result['itemitem3'] = result['user_id'].apply(lambda user_id: [id_to_itemid[rec[0]]  
                                                              for rec in model.recommend(userid=userid_to_id[user_id], 
                                                                user_items=sparse_user_item,   # на вход user-item matrix
                                                                N=3, 
                                                                filter_already_liked_items=False, 
                                                                filter_items=None, 
                                                                recalculate_user=True)
                                                                             ])

In [ ]:
result['itemitem1'] = result['user_id'].apply(lambda user_id: [id_to_itemid[rec[0]]  
                                                              for rec in model.recommend(userid=userid_to_id[user_id], 
                                                                user_items=sparse_user_item,   # на вход user-item matrix
                                                                N=1, 
                                                                filter_already_liked_items=False, 
                                                                filter_items=None, 
                                                                recalculate_user=True)
                                                                             ])

In [ ]:
def precision_at_k2(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [ ]:
result

,user_id,actual,random_recommendation_wight,random_recommendation_5000,itemitem5,itemitem3,itemitem1
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[995965, 1134688, 1115098, 1124029, 848071]","[856252, 1022003, 1070803, 6534178, 1068708]","[1082185, 981760, 1127831, 995242, 840361]","[1082185, 981760, 1127831]",[1082185]
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1062002, 845208, 908318, 906844, 6534178]","[951590, 6533765, 918368, 1056492, 845307]","[1082185, 981760, 1098066, 826249, 995242]","[1082185, 981760, 1098066]",[1082185]
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1139901, 6534178, 1036249, 12731419, 16809497]","[901067, 1011300, 7168058, 1014732, 1025611]","[1082185, 981760, 1127831, 995242, 1098066]","[1082185, 981760, 1127831]",[1082185]
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[911622, 952163, 917033, 992986, 949835]","[9365106, 6534178, 951164, 13381928, 6533765]","[1082185, 981760, 1127831, 995242, 1098066]","[1082185, 981760, 1127831]",[1082185]
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[988202, 1056509, 982009, 1135917, 872342]","[5569327, 6534178, 6034964, 1011312, 863505]","[1082185, 981760, 1098066, 995242, 1127831]","[1082185, 981760, 1098066]",[1082185]
...,...,...,...,...,...,...,...
2037,2496,[6534178],"[13157872, 10456346, 9526563, 9245106, 895163]","[9673980, 1021324, 12262778, 914754, 956672]","[1082185, 981760, 1098066, 826249, 995242]","[1082185, 981760, 1098066]",[1082185]
2038,2497,"[1016709, 9835695, 1132298, 16809501, 845294, ...","[5567030, 1097195, 909750, 943335, 837304]","[1139216, 1007136, 905582, 830203, 824311]","[1082185, 981760, 1098066, 995242, 826249]","[1082185, 981760, 1098066]",[1082185]
2039,2498,"[15716530, 834484, 901776, 914190, 958382, 972...","[6633167, 1076954, 5572741, 1121961, 915174]","[10198378, 9245106, 871756, 1000753, 899019]","[1082185, 981760, 1098066, 1127831, 995242]","[1082185, 981760, 1098066]",[1082185]
2040,2499,"[867188, 877580, 902396, 914190, 951590, 95813...","[1103295, 1125891, 1081029, 842762, 1104414]","[6534178, 7462562, 1023720, 866211, 12301109]","[1082185, 981760, 1098066, 826249, 995242]","[1082185, 981760, 1098066]",[1082185]


In [ ]:
for name_col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k2(row[name_col], row['actual']), axis=1).mean(),4)}:{name_col}")

1.0:actual
0.0207:random_recommendation_wight
0.0333:random_recommendation_5000
0.1541:itemitem5
0.1923:itemitem3
0.3477:itemitem1
